# 🍏 Health Resource Search Agent Tutorial 🍎

Welcome to the **Health Resource Search Agent** tutorial! We'll use **Azure AI Foundry** SDKs to build an assistant that can:

1. **Upload** health and recipe files into a vector store.
2. **Create an Agent** with a **File Search** tool.
3. **Search** these documents for relevant dietary info.
4. **Answer** health and wellness questions (with disclaimers!).

## ⚠️ Important Medical Disclaimer ⚠️

>   **All health information in this notebook is for general educational purposes only and is not a substitute for professional medical advice, diagnosis, or treatment**. Always seek the advice of a qualified healthcare professional with any questions you may have.

## Prerequisites
- Complete Agent basics notebook - [1-basics.ipynb](1-basics.ipynb)
- **Roles**
  - **Azure AI Developer** on your Azure AI Foundry project.
  - **Storage Blob Data Contributor** on the project’s Storage account.
  - If standard agent setup is used with your own Search resource, also ensure you have **Cognitive Search Data Contributor** on that resource.

# Let's Get Searching!

We'll show you how to upload some sample files, create a vector store for them, then spin up an agent that can search these resources for dietary guidelines, recipes, and more. Enjoy!

<img src="./seq-diagrams/3-file-search.png" width="800"/>

# 🔐 Authentication Setup

Before running the next cell, make sure you're authenticated with Azure CLI. Run this command in your terminal:

```az login --use-device-code```

This will provide you with a device code and URL to authenticate in your browser, which is useful for:

- Remote development environments
- Systems without a default browser
- Corporate environments with strict security policies

After successful authentication, you can proceed with the notebook cells below.

# 1. Initial Setup

Here we import needed libraries, load environment variables from .env, and initialize our AIProjectClient. Let's do this! 🎉

In [ ]:
#pragma warning disable OPENAI001

#r "nuget: Azure.Identity, 1.18.0-beta.2"
#r "nuget: Azure.AI.Projects, 1.2.0-beta.5"
#r "nuget: dotenv.net"

using System;
using System.Text;
using System.Globalization;
using System.IO;
using System.ClientModel.Primitives;
using System.Reflection;
using Azure.Identity;
using Azure.AI.Projects;
using Azure.AI.Projects.OpenAI;
using OpenAI.Responses;
using OpenAI.Files;
using OpenAI.VectorStores;
using dotenv.net;  

DotEnv.Load(new DotEnvOptions(envFilePaths: new[] { Path.Combine(".","..", ".env") })); 

In [ ]:
var tenantId = Environment.GetEnvironmentVariable("TENANT_ID");
var projectEndpoint = Environment.GetEnvironmentVariable("AI_FOUNDRY_PROJECT_ENDPOINT");
var modelDeployment = Environment.GetEnvironmentVariable("MODEL_DEPLOYMENT_NAME");
var agentName = "health-resource-agent"; 
Console.WriteLine($"🔑 Using Tenant ID: {tenantId}");

AIProjectClient projectClient;

try
{
    var credential = new DefaultAzureCredential(new DefaultAzureCredentialOptions
    {
        TenantId = tenantId
    });

    projectClient = new AIProjectClient(new Uri(projectEndpoint), credential);
    Console.WriteLine("✅ Successfully initialized AIProjectClient");
}
catch (Exception ex)
{
    Console.WriteLine($"❌ Error initializing AIProjectClient: {ex.Message}");
    throw;
}

# 2. Prepare Sample Files 🍲🗒

We'll create some dummy .md files (for recipes and guidelines). Then we'll store them in a vector store for searching.

In [ ]:
async Task<string[]> CreateSampleFiles()
{

    string recipesMarkdown = @"
        # Healthy Recipes Database
        ## Gluten-Free Recipes
        1. Quinoa Bowl
           - Ingredients: quinoa, vegetables, olive oil
           - Instructions: Cook quinoa, add vegetables

        2. Rice Pasta with Vegetables
           - Ingredients: rice pasta, mixed vegetables
           - Instructions: Boil pasta, sauté vegetables

        ## Diabetic-Friendly Recipes
        1. Low-Carb Stir Fry
           - Ingredients: chicken, vegetables, tamari sauce
           - Instructions: Cook chicken, add vegetables

        2. Greek Salad
           - Ingredients: cucumber, tomatoes, feta, olives
           - Instructions: Chop vegetables, combine

        ## Heart-Healthy Recipes
        1. Baked Salmon
           - Ingredients: salmon, lemon, herbs
           - Instructions: Season salmon, bake
        2. Mediterranean Bowl
           - Ingredients: chickpeas, vegetables, tahini
           - Instructions: Combine ingredients
    ";

    string guidelinesMarkdown = @"
        # Dietary Guidelines

        ## General Guidelines
        - Eat a variety of foods
        - Control portion sizes
        - Stay hydrated

        ## Special Diets
        1. Gluten-Free Diet
           - Avoid wheat, barley, rye
           - Focus on naturally gluten-free foods

        2. Diabetic Diet
           - Monitor carbohydrate intake
           - Choose low glycemic foods
        3. Heart-Healthy Diet
           - Limit saturated fats
           - Choose lean proteins
    ";

   var recipesFilename = Environment.GetEnvironmentVariable("RECIPES_FILENAME") ?? "recipes.md";
   var guidelinesFilename = Environment.GetEnvironmentVariable("GUIDELINES_FILENAME") ?? "guidelines.md";

   await File.WriteAllTextAsync(recipesFilename, recipesMarkdown);
   await File.WriteAllTextAsync(guidelinesFilename, guidelinesMarkdown);

    Console.WriteLine($"📄 Created sample resource files: {recipesFilename}, {guidelinesFilename}");
    return new[] { recipesFilename, guidelinesFilename };
} 

var sampleFiles = await CreateSampleFiles();

### ✨ Note on Search Permissions

When creating the vector store, you must also have Cognitive Search Data Contributor role on your Azure AI Search resource (if you're using the standard agent setup with your own Search resource). Missing this role will often cause a Forbidden error. See Authentication Setup for details on configuring permissions.

# 3. Create a Vector Store 📚

We'll upload our newly created files and group them into a single vector store for searching. This is how the agent can later find relevant text.

In [ ]:
#pragma warning disable OPENAI001

async Task<(VectorStore, string[])> CreateVectorStore(string[] files, string vectorStoreName = "my_health_resources")
{
    try
    {
        List<string> fileIds = new ();
        VectorStoreCreationOptions options = new ()
        {
            Name = vectorStoreName
        };

        foreach(var file in files)
        {
            var uploadedFile = await projectClient.OpenAI.Files.UploadFileAsync(file,FileUploadPurpose.Assistants);
            options.FileIds.Add(uploadedFile.Value.Id);
            fileIds.Add(uploadedFile.Value.Id);
            Console.WriteLine($"⬆️ Uploaded file: {file} with ID: {uploadedFile.Value.Id}");
        }

        var vectorStore = await projectClient.OpenAI.VectorStores.CreateVectorStoreAsync(options);
        Console.WriteLine($"🎉 Created vector store '{vectorStoreName}', ID: {vectorStore.Value.Id}");
        return (vectorStore.Value, fileIds.ToArray());
    }
    catch (Exception ex)
    {
        Console.WriteLine($"❌ Error creating vector store: {ex.Message}");
        throw;
    }
}

VectorStore vectorStore;
string[] uploadedFileIds;
if(sampleFiles.Length > 0)
{
    (vectorStore, uploadedFileIds) = await CreateVectorStore(sampleFiles);
}
else
{
     Console.WriteLine("⚠️ No sample files available - please run the previous cell to create sample files first");
}     

# 4. Create the Health Resource Agent 🔎

We use a **FileSearchTool** pointing to our newly created vector store, then create the Agent with instructions about disclaimers, dietary help, etc.

In [ ]:
#pragma warning disable OPENAI001

async Task<AgentVersion> CrateHealthResourceAgent(string vectoreStoreId)
{
    try
    {
        PromptAgentDefinition agentDefinition = new(modelDeployment)
        {
            Instructions = @"
                You are a health resource advisor with access to dietary and recipe files.
                    You:
                    1. Always present disclaimers (you're not a doctor!)
                    2. Provide references to the files when possible
                    3. Focus on general nutrition or recipe tips.
                    4. Encourage professional consultation for more detailed advice.
            ",
            Tools = {
                ResponseTool.CreateFileSearchTool([vectoreStoreId])
            }
        };
        var agent = await projectClient.Agents.CreateAgentVersionAsync(
            agentName: agentName, 
            options: new(agentDefinition)
        );
        Console.WriteLine($"🎉 Created health resource agent, ID: {agent.Value.Id}");
        Console.WriteLine("📋 Vector store will be attached at message level for better compatibility");
        return agent.Value;
    }
    catch (Exception ex)
    {
        Console.WriteLine($"❌ Error creating agent: {ex.Message}");
        return null;
    }
}

AgentVersion healthAgent;

if(vectorStore != null)
{
    if(healthAgent != null)
    {
        try
        {
            await projectClient.Agents.DeleteAgentAsync(agentName);
            Console.WriteLine($"🗑️ Deleted existing agent: {agentName}");
        }
        catch (Exception ex)
        {
            Console.WriteLine($"❌ Error deleting previous agent: {ex.Message}");
        }
    }
    
    healthAgent = await CrateHealthResourceAgent(vectorStore.Id);
}

# 5. Searching Health Resources 🏋️👩‍🍳

We'll create a new conversation thread and ask queries like “Gluten-free recipe ideas?” or “Heart-healthy meal plan?” The agent will do file search on the vector store to find relevant info.

In [ ]:
async Task<ProjectConversation> CreateSearchConversation(AgentVersion agent)
{
    try
    {
        var conversation = await projectClient.OpenAI.Conversations.CreateProjectConversationAsync();   
        Console.WriteLine($"📝 Created new search conversation, ID: {conversation.Value.Id}");
        return conversation.Value;
    }
    catch (Exception ex)
    {
        Console.WriteLine($"❌ Error creating conversation: {ex.Message}");
        return null;
    }
}

In [ ]:
#pragma warning disable OPENAI001

async Task<ResponseResult> AskSearchQuestion(string conversationId, AgentVersion agent, string userQuestion)
{
    try
    {
        var responsesClient = projectClient.OpenAI.GetProjectResponsesClientForAgent(
                    defaultAgent: healthAgent,
                    defaultConversationId: conversationId);
        var responseResult = await responsesClient.CreateResponseAsync(userQuestion);
        Console.WriteLine($"🤖 Simple run finished with status: {responseResult.Value.Status}");        
        return responseResult.Value;
    }
    catch (Exception ex)
    {
        Console.WriteLine($"❌ Error asking question: {ex.Message}");
        return null;
    }
}

In [ ]:
ProjectConversation searchConversation = null;
if(healthAgent != null)
{
    searchConversation = await CreateSearchConversation(healthAgent);
    var queries = new[]
    {
        "Could you suggest a gluten-free lunch recipe?",
        "Show me some heart-healthy meal ideas.",
        "What guidelines do you have for someone with diabetes?"
    };
    foreach(var query in queries)
    {
        var response = await AskSearchQuestion(searchConversation.Id, healthAgent, query);
    }
}

# 6. View Results & Citations 📄

We'll read the conversation thread to see how the agent responded and see if it cited the correct files.

In [ ]:
#pragma warning disable OPENAI001

async Task DisplayConversationMessages(string conversationId)
{
    try
    {
        Console.WriteLine($"🗣️ Conversation so far:");
        Console.WriteLine(new string('=', 60));
        int citationCount = 0;
        await foreach (AgentResponseItem item in projectClient.OpenAI.Conversations.GetProjectConversationItemsAsync(
                        conversationId, order:"asc"
                       ))
        {
            var responseResultItem = item.AsResponseResultItem();
            
            if(responseResultItem is MessageResponseItem messageResponseItem)
            {
                var content = messageResponseItem.Content?[0];
                var role = messageResponseItem.Role.ToString();
                var roleEmoji = role == "User" ? "👤" : "🤖";
                Console.WriteLine($"{roleEmoji} {role}: {content?.Text}");
                if(content.OutputTextAnnotations?.Count > 0)
                {
                    foreach(var annotation in content.OutputTextAnnotations)
                    {
                        if(annotation is FileCitationMessageAnnotation)
                        {
                            citationCount++;
                            var fileAnnotation = annotation as FileCitationMessageAnnotation;
                            var fileId = fileAnnotation.FileId;
                            var filename = fileAnnotation.Filename;
                            Console.WriteLine($"\n📄 Citation {citationCount}: Filename: {filename}, File ID: {fileId}");
                        }
                    }
                }
                Console.WriteLine(new string('-', 40));
            }
        }
        if(citationCount == 0)
        {
            await foreach (AgentResponseItem item in projectClient.OpenAI.Conversations.GetProjectConversationItemsAsync(
                        conversationId, order:"asc"
                       ))
            {
                var responseResultItem = item.AsResponseResultItem();
                if(responseResultItem is MessageResponseItem messageResponseItem)
                {
                    var contentText = messageResponseItem.Content?[0].Text;
                    if (new[] { "quinoa", "salmon", "gluten-free", "diabetic", "heart-healthy" }.Any(keyword => contentText.Contains(keyword)))
                    {
                        Console.WriteLine("✅ Agent appears to be using file content (found relevant keywords)");
                        break;
                    }
                }
            }
        }
    }
    catch (Exception ex)
    {
        Console.WriteLine($"❌ Error displaying messages: {ex.Message}");
    }
}

if(searchConversation != null)
{
    await DisplayConversationMessages(searchConversation.Id);
}
else
{
    Console.WriteLine("⚠️ No search thread available to display.");
    Console.WriteLine("💡 Please run the previous cell first to create a search thread and ask questions.");
    Console.WriteLine("🔄 Make sure the health agent was created successfully before proceeding.");
}

# 7. Cleanup & Best Practices 🧹

We'll optionally remove the vector store, the uploaded files, and the agent. In a production environment, you might keep them around longer. Meanwhile, here are some tips:

1. Resource Management

    - Keep files grouped by category, regularly prune old or irrelevant files.
    - Clear out test agents or vector stores once you're done.

2. Search Queries

    - Provide precise or multi-part queries.
    - Consider synonyms or alternative keywords ("gluten-free" vs "celiac").

3. Health Information
    - Always disclaim that you are not a medical professional.
    - Encourage users to see doctors for specific diagnoses.

4. Performance

    - Keep an eye on vector store size.
    - Evaluate search accuracy with azure-ai-evaluation!

In [ ]:
async Task CleanUpAll()
{
    try
    {
        if(vectorStore != null)
        {
            await projectClient.OpenAI.VectorStores.DeleteVectorStoreAsync(vectorStore.Id);
            Console.WriteLine($"🗑️ Deleted vector store: {vectorStore.Id}");
        }

        if(uploadedFileIds != null)
        {
            foreach(var fileId in uploadedFileIds)
            {
                await projectClient.OpenAI.Files.DeleteFileAsync(fileId);
                Console.WriteLine($"🗑️ Deleted uploaded file: {fileId}");
            }
        }

        if(healthAgent != null)
        {
            await projectClient.Agents.DeleteAgentAsync(agentName);
            Console.WriteLine($"🗑️ Deleted agent: {agentName}");
        }

        if(sampleFiles != null)
        {
            foreach(var file in sampleFiles)
            {
                if(File.Exists(file))
                {
                    File.Delete(file);
                    Console.WriteLine($"🗑️ Deleted sample file: {file}");
                }
            }
        }
    }
    catch (Exception ex)
    {
        Console.WriteLine($"❌ Error during cleanup: {ex.Message}");
    }
}

await CleanUpAll();

# Congratulations! 🎉

You've successfully completed the Health Resource Search Agent tutorial! Here's what was accomplished:

## ✅ What We Built

### 🔍 File Search Agent
- Created an AI agent with file search capabilities
- Enabled the agent to search through uploaded health documents using semantic search
- Configured health-focused instructions with appropriate medical disclaimers

### 📚 Key Features Demonstrated

1. 📄 File Upload & Vector Store Creation

    - Uploaded health resource files to Azure AI service
    - Created a vector store for semantic document search
    
2. 🔎 Semantic Document Search

    - Agent successfully searched through health and recipe documents
    - Provided relevant answers based on uploaded file contents

3. 🏥 Health-Focused Responses

    - Agent answered questions about gluten-free recipes, heart-healthy meals, and diabetic guidelines
    - Provided responsible AI disclaimers about medical advice
    - Referenced content from uploaded documents

4. 🧹 Resource Management

    - Properly cleaned up vector stores, files, and agents
    - Demonstrated best practices for resource lifecycle management

## 🎯 Key Concepts Learned

- **File Search Tool**: Enables agents to search through uploaded documents using semantic search
- **Vector Stores**: Convert documents into searchable numerical vectors
- **Message-Level Attachments**: More reliable than agent-level tool_resources for current SDK version
- **Semantic Search**: Agents can find relevant content even when exact words don't match
- **Responsible AI**: Always include medical disclaimers for health-related content

## 🚀 What's Next?

Continue your Azure AI Agent Service journey with these advanced topics:

- [4-bing_grounding.ipynb](./4-bing_grounding.ipynb) - Agents with real-time web search capabilities
- [5-agents-aisearch.ipynb](./5-agents-aisearch.ipynb) - Integration with Azure AI Search for enterprise knowledge
- [6-agents-az-functions.ipynb](./6-agents-az-functions.ipynb) - Agents that can trigger Azure Functions and workflows

## 💡 Best Practices Recap

- **File Attachment Strategy** - Use message-level file attachments instead of agent-level tool_resources
- **API Structure** - Use the nested client structure (agents.files, agents.threads, etc.)
- **Vector Store Management** - Keep documents organized and clean up when done
- **Health Content** - Always include medical disclaimers for health-related responses
- **Resource Cleanup** - Delete agents, vector stores, and files to manage costs
- **Semantic Search** - Leverage vector stores for intelligent document search

Ready to explore more advanced file search and integration patterns? Let's continue! 🚀

Happy (healthy) searching! 🔍💚